<a href="https://colab.research.google.com/github/diputs03/AI-Studies/blob/main/Creating_network/dymamic_architect_rebuilt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Aiming a Dynaimic Graph-structured NeuronNetwork
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sympy

In [ ]:
"""
Activation function in this case in tanh
"""

def softmax(Z):
  return np.exp(Z) / np.sum(np.exp(Z))
def d_tanh(Z):
  return 1 - np.square(np.tanh(Z))

class Model:
  class Neuron:
    def __init__(self, name, prev, next):
      self.name, self.prev, self.next = name, prev, next
  def __init__(self, input_size, output_size):
    self.Input_layer = [Model.Neuron(f"input{i}", [], []) for i in range(input_size)]
    self.Output_layer = [Model.Neuron(f"output{o}", [], []) for o in range(output_size)]
    self.Input_layer[i].next = self.Output_layer for i in range(input_size)
    self.Output_layer[o].prev = self.Input_layer for o in range(output_size)
  def forward(self, X, batch_size):
    assert X[i].shape() == batch_size for i in range()